In [1]:
from classes import *
from helper_functions import *

import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time
import random
random.seed(2022)

# for configurations
from dotenv import load_dotenv
import sys, os, re, math
load_dotenv()

# for distance matrix
import gmaps
import googlemaps
from haversine import haversine
from scipy.spatial.distance import cdist

# for routing
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import warnings
warnings.filterwarnings('ignore')

In [2]:
file_path = 'data/processed_data.csv'
num_vehicles = 5
ins_ends_coords = [(-34.8218243,138.7292797),(-34.8104796,138.6111791),(-34.8938435,138.6918266),(-34.7825552,138.610732),(-34.8516117,138.6722955)]

In [3]:
# Importing the dataset and configuration parameters
data = pd.read_csv(file_path)
API_KEY = os.getenv("API_KEY")
FACTORY_GEO_COORD = os.getenv("FACTORY_GEO_COORD") 
factory_coord = list(map(float,FACTORY_GEO_COORD.split(',')))

# enter your api key here
gmaps.configure(api_key=API_KEY)

In [4]:
## [[DATA HANDLING AND MANIPULATION]]
jobs_installer_1 = data[data["installers_required"]==1].reset_index(drop=True)
df_pending = jobs_installer_1[['id','Latitude','Longitude']]
# Inserting factory location to top
new_row = pd.DataFrame({'id':0, 'Latitude':factory_coord[0],'Longitude':factory_coord[1]}, index =[0])
df_pending = pd.concat([new_row, df_pending]).reset_index(drop=True)
df_pending.set_index('id', inplace=True)
df_pending['Latitude'] = df_pending['Latitude'].astype(float)
df_pending['Longitude'] = df_pending['Longitude'].astype(float)

In [5]:
for i in range(len(ins_ends_coords)):
    ins_id = i+1
    df_pending.loc[ins_id] = [ins_ends_coords[i][0],ins_ends_coords[i][1]]

In [6]:
jobs, ins_ends = [],[]
for i, row in df_pending.iterrows():
    if i == 0:
        continue
    elif i//100 == 0: # Expecting max 100 installers for the program and job_ids no less than 100
        ins_end = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
        ins_ends.append(ins_end)
    else:
        job = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
        jobs.append(job)

In [7]:
factory = {'location': (factory_coord[0],factory_coord[1])}
factory_layer = gmaps.symbol_layer([factory['location']], hover_text='Factory', info_box_content='Factory', fill_color='white', stroke_color='red', scale=6)

In [8]:
job_locations = [job['location'] for job in jobs]
job_labels = [job['id'] for job in jobs]
jobs_layer = gmaps.symbol_layer(
    job_locations, hover_text=job_labels, fill_color='white', stroke_color='black', scale=3
)

In [9]:
ins_end_locations = [ins_end['location'] for ins_end in ins_ends]
ins_labels = [ins_end['id'] for ins_end in ins_ends]
ins_ends_layer = gmaps.symbol_layer(
    ins_end_locations, hover_text=ins_labels, fill_color='white', stroke_color='red', scale=3
)

In [10]:
fig = gmaps.figure()
fig.add_layer(factory_layer)
fig.add_layer(jobs_layer)
fig.add_layer(ins_ends_layer)
# fig

In [11]:
demands = [0]
for i in range(len(jobs)):
    if np.isnan(jobs_installer_1.loc[i,'expected_job_time']):
        demands.append(int(60))
        continue
    demands.append(int(jobs_installer_1.loc[i,'expected_job_time']))

In [12]:
end_locations = []
for i in range(num_vehicles):
    end_locations.append(len(jobs)+i+1)

In [13]:
dist_matrix,time_matrix = get_distance_time_matrices(df_pending)

In [14]:
routes,total_distance,total_load = solve_vrp_for(time_matrix, num_vehicles, demands, end_locations)

TypeError: solve_vrp_for() missing 5 required positional arguments: 'end_locations', 'pref_dates', 'pref_days', 'pref_installers', and 'pref_time_windows'

In [ ]:
def map_solution(factory, jobs, routes):
    colors = ['red','yellow','green','blue','#b100cd']
    for vehicle_id in routes:
        waypoints = []
        # skip depot (occupies first and last index)
        for job_index in routes[vehicle_id][1:-1]: 
            waypoints.append(jobs[job_index-1]['location'])
        installer_end_location = ins_ends[vehicle_id]['location']
        if len(waypoints) == 0:
            print('Empty route:', vehicle_id)
        else:
            route_layer = gmaps.directions_layer(
                factory['location'], waypoints[-1], waypoints=waypoints[0:-1], show_markers=False,
                stroke_color=colors[vehicle_id], stroke_weight=3, stroke_opacity=0.5)
            fig.add_layer(route_layer)
            
            # complete the route from last shipment to depot
            return_layer = gmaps.directions_layer(
                waypoints[-1], installer_end_location, show_markers=False,
                stroke_color=colors[vehicle_id], stroke_weight=3, stroke_opacity=0.5)
            fig.add_layer(return_layer)

In [ ]:
if routes:
    map_solution(factory, jobs, routes)
else:
    print('No solution found.') 

fig

In [ ]:
## GOOGLE DISTANCE MATRIX API
cumul_dist,cumul_time=0,0
for route in routes:
    total_distance,total_time = 0,0
    prev_node = None
    curr_node = None
    for node in routes[route]:
        if prev_node==None:
            prev_node = node
            continue
        curr_node = node
        prev_coord = (df_pending.iloc[prev_node]['Latitude'],df_pending.iloc[prev_node]['Longitude'])
        curr_coord = (df_pending.iloc[curr_node]['Latitude'],df_pending.iloc[curr_node]['Longitude'])
        total_distance += get_distance(prev_coord,curr_coord)
        total_time += get_travel_time(prev_coord,curr_coord)
#         print(prev_coord, end='--')
        prev_node = curr_node
#     print(curr_coord)
    print(total_distance/1000,total_time)
    cumul_dist+=total_distance
    cumul_time+=total_time
print(cumul_dist/1000,cumul_time)

In [ ]:
# route_1 = routes[0]
# optimized_path = []
# for i in route_1:
# #     print(df_pending.iloc[i,:].Latitude)
#     optimized_path.append((df_pending.iloc[i,:].Latitude,df_pending.iloc[i,:].Longitude))

In [ ]:
# for i in optimized_path:
#     print(str(i[0])+','+str(i[1]),end=',\n')
# time_matrix[0][46],time_matrix[7][46],time_matrix[7][93],time_matrix[93][38],time_matrix[38][102],time_matrix[102][50],time_matrix[50][16],time_matrix[16][94],time_matrix[94][108]

In [ ]:
# planned_ids=[]
# for route in routes:
#     for node in routes[route]:
#         if df_pending.index[node]//100 != 0:
#             planned_ids.append(df_pending.index[node])

In [ ]:
routes

In [ ]:
# for route in routes:
#     sum=0
#     for j in range(len(routes[route])):
#         try:
#             node_1 =routes[route][j]
#             node_2 = routes[route][j+1]
#             sum+=time_matrix[node_1][node_2]
#         except:
#             print(sum)
#             continue